In [4]:
import datetime
import numpy as np
import pandas as pd

In [5]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [6]:
import plotly.graph_objs as go
import plotly.io as pio

In [1]:
#from openbb_terminal.sdk import openbb

In [2]:
from sklearn.decomposition import PCA

In [3]:
pio.templates.default="plotly"

In [4]:
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [5]:
ticker_field= [
    html.Label("Enter Ticker Symbols:"),
    dcc.Input(
    id="ticker-input",
    type="text",
    ),
]

In [6]:
components_field=[
    html.Label("Select Number of Components:"),
    dcc.Dropdown(
        id="component-dropdown",
        options=[{"label":i,"value":i} for i in
range(1,6)],
        value=3,
    ),
]

In [7]:
date_picker_field=[
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id="date-picker",
    start_date=datetime.datetime.now()-
datetime.timedelta(365*3),
    end_date=datetime.datetime.now(),
    display_format="YYY-MM-DD"
    ),
]
submit=[
    html.Button("Submit",id="submit-button"),
]

In [10]:
 app.layout=dbc.Container(
 [
     html.H1("PCS on STock Returns"),
     #Ticker input
     dbc.Row([dbc.Col(ticker_field)]),
     dbc.Row([dbc.Col(components_field)]),
     dbc.Row([dbc.Col(date_picker_field)]),
     dbc.Row([dbc.Col(submit)]),
     
     #Charts
     dbc.Row(
        [
            dbc.Col([dcc.Graph(id="bar-chart")],
width=4),
            dbc.Col([dcc.Graph(id="line-chart")],
width=4),
        ]  
     ),
    ]
  )

In [ ]:
@app.callback(
    [
        Output("bar-chart","figure"),
        Output("line-chart","figure"),
    ],
    [Input("submit-button","n_clicks")],
    [
        dash.dependencies.State("ticker-input","value")
    ]
)